# Initial Normalization of Images

In [9]:
from PIL import Image, ImageEnhance
import os
from tqdm import tqdm
import cv2

In [2]:
def normalize_images(dataset_dir, dir, new_dir, img):
    image = Image.open(os.path.join(dataset_dir, dir,img))
    new_image = image.resize((256, 256))
    new_image.save(os.path.join(dataset_dir, new_dir, img))

Resize Landscape images to 64x64

In [6]:
for i in tqdm(os.listdir('../dataset/landscape_dat/landscapes')):
    normalize_images('../dataset/landscape_dat', 'landscapes', 'landscape_256x256', i)

100%|██████████| 4319/4319 [02:26<00:00, 29.52it/s]


In [7]:
def color_to_bw(dataset_dir, dir, new_dir, img):
    image = Image.open(os.path.join(dataset_dir, dir, img))
    image_new = image.convert('L')
    image_new.save(os.path.join(dataset_dir, new_dir, img))

In [8]:
for i in tqdm(os.listdir('../dataset/landscape_dat/landscape_256x256')):
    color_to_bw('../dataset/landscape_dat', 'landscape_256x256', 'landscape_256x256_bw', i)

100%|██████████| 4319/4319 [00:39<00:00, 109.05it/s]


# Run each Model

# Testing and Creating Dataset

In [1]:
cd

C:\Users\ivann


c:\Users\ivann\anaconda3\envs\deep_learning\Lib\site-packages\IPython\core\magics\osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
c:\Users\ivann\anaconda3\envs\deep_learning\Lib\site-packages\IPython\core\magics\osm.py:428: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
cd "Documents\School\8vo. Semestre\Deep Learning"

C:\Users\ivann\Documents\School\8vo. Semestre\Deep Learning


c:\Users\ivann\anaconda3\envs\deep_learning\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
import cv2
import csv
from skimage.color import rgb2lab
import numpy as np
from tqdm import tqdm
import os

def mse(imageA, imageB):
    # Mean Squared Error calculation
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])
    return err

def total_mse_2_images(image1_path, image_2_path):
    # Load images
    image1 = cv2.imread(image1_path)
    image2 = cv2.imread(image_2_path)

    # Convert images to LAB space
    image1_lab = rgb2lab(image1)
    image2_lab = rgb2lab(image2)

    # Split LAB images into L, A, B channels
    L1, A1, B1 = cv2.split(image1_lab)
    L2, A2, B2 = cv2.split(image2_lab)

    # Calculate MSE for each channel
    mse_A = mse(A1, A2)
    mse_B = mse(B1, B2)

    return mse_A, mse_B

original = os.listdir('Dataset/dataset/imagenet/imagenet_64x64/')
original = ['Dataset/dataset/imagenet/imagenet_64x64/' + org for org in original][:18379]
infer = os.listdir('Image_Processing/Image_Classifier/zhang_2016/results_total/imagenet/results/') # Just to get outputs names
models = ['ddcolor', 'larsson_2016', 'siggraph_2016', 'zhang_2016']


In [4]:
with open('Image_Processing/Image_Classifier/dataset_64x64.csv', 'a', newline='') as csvfile:
    fieldnames = ['datatype', 'image', 'model']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    for org, inf in zip(original, infer):
        
        mse_total_A = []
        mse_total_B = []
        
        for data in models:
            inf_complete = f'Image_Processing/Image_Classifier/{data}/results_total/imagenet/results/'+ inf
            #try:
            mse_A, mse_B = total_mse_2_images(org, inf_complete)
            #except:
            #    mse_A, mse_B = 1000, 1000
            mse_total_A.append(mse_A)
            mse_total_B.append(mse_B)
        
        mse_total_A = np.array(mse_total_A)
        mse_total_B = np.array(mse_total_B)
        mse_total = (mse_total_A + mse_total_B)/2

        model = models[np.argmin(mse_total)]
        
        print(inf,model)
        writer.writerow({'datatype':'imagenet', 'image':inf, 'model':model})

image_00000.jpg siggraph_2016
image_00001.jpg larsson_2016
image_00002.jpg larsson_2016
image_00003.jpg larsson_2016
image_00004.jpg ddcolor
image_00005.jpg zhang_2016
image_00006.jpg ddcolor
image_00007.jpg siggraph_2016
image_00008.jpg ddcolor
image_00009.jpg siggraph_2016
image_00010.jpg larsson_2016
image_00011.jpg ddcolor
image_00012.jpg siggraph_2016
image_00013.jpg ddcolor
image_00014.jpg zhang_2016
image_00015.jpg zhang_2016
image_00016.jpg ddcolor
image_00017.jpg zhang_2016
image_00018.jpg ddcolor
image_00019.jpg larsson_2016
image_00020.jpg larsson_2016
image_00021.jpg larsson_2016
image_00022.jpg ddcolor
image_00023.jpg ddcolor
image_00024.jpg ddcolor
image_00025.jpg siggraph_2016
image_00026.jpg siggraph_2016
image_00027.jpg larsson_2016
image_00028.jpg siggraph_2016
image_00029.jpg larsson_2016
image_00030.jpg ddcolor
image_00031.jpg ddcolor
image_00032.jpg ddcolor
image_00033.jpg siggraph_2016
image_00034.jpg siggraph_2016
image_00035.jpg siggraph_2016
image_00036.jpg lar

: 

In [5]:
# Testing single image

mse_total_A = []
mse_total_B = []

org = '../dataset/landscape_dat/landscape_256x256/00000000_(2).jpg'

print(org)

for model in models:
    inf_complete = f'{model}/results_256x256/00000000_(2).jpg'
    try:
        mse_A, mse_B = total_mse_2_images(org, inf_complete)
    except:
        mse_A, mse_B = 1000, 1000
    mse_total_A.append(mse_A)
    mse_total_B.append(mse_B)

print(mse_total_A)
print(mse_total_B)

models = ['ddcolor', 'larsson_2016', 'siggraph_2016', 'zhang_2016']

../dataset/landscape_dat/landscape_256x256/00000000_(2).jpg
[1000, 1000, 1000, 1000]
[1000, 1000, 1000, 1000]


# TinyImageNet

In [24]:
cd ..

c:\Users\ivann\Documents\School\8vo. Semestre\Deep Learning


In [26]:
from datasets import load_dataset
import io
from PIL import Image
import numpy as np
from tqdm import tqdm

tiny_imagenet = load_dataset('Maysee/tiny-imagenet', split='train')
for i in tqdm(range(len(tiny_imagenet))):
    tiny_imagenet[i]['image'].save(f'Dataset/dataset/imagenet/imagenet_64x64/image_{i:05}.jpg')
    
print(len(tiny_imagenet))

#tiny_imagenet[0]['image'].save()

100%|██████████| 100000/100000 [01:34<00:00, 1053.76it/s]

100000


In [27]:
def color_to_bw(dataset_dir, dir, new_dir, img):
    image = Image.open(os.path.join(dataset_dir, dir, img))
    image_new = image.convert('L')
    image_new.save(os.path.join(dataset_dir, new_dir, img))

In [28]:
for i in tqdm(os.listdir('Dataset/dataset/imagenet/imagenet_64x64')):
    color_to_bw('Dataset/dataset/imagenet', 'imagenet_64x64', 'imagenet_64x64_bw', i)

100%|██████████| 100000/100000 [01:35<00:00, 1045.21it/s]


# Change Metric from MSE to CSPNR

In [1]:
cd

C:\Users\ivann


c:\Users\ivann\anaconda3\envs\deep_learning\Lib\site-packages\IPython\core\magics\osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
c:\Users\ivann\anaconda3\envs\deep_learning\Lib\site-packages\IPython\core\magics\osm.py:428: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
cd "Documents\School\8vo. Semestre\Deep Learning"

C:\Users\ivann\Documents\School\8vo. Semestre\Deep Learning


c:\Users\ivann\anaconda3\envs\deep_learning\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [42]:
import cv2
import csv
from skimage.color import rgb2lab
import numpy as np
from tqdm import tqdm
import os
import math

def total_CPSNR_2_images(image1_path, image_2_path):
    # Load images
    image1 = cv2.imread(image1_path)
    image2 = cv2.imread(image_2_path)

    mse = (np.sum(np.square(image1-image2)))/(3*image1.shape[0]*image1.shape[1])
    div = (255**2)/(mse)
    cpsnr = 10*math.log10(div)

    return cpsnr

#original = os.listdir('Dataset/dataset/landscape_dat/landscape_64x64/')
original = os.listdir('Dataset/dataset/imagenet/imagenet_64x64/')
#original = ['Dataset/dataset/landscape_dat/landscape_64x64/' + org for org in original]#[:18379]
original = ['Dataset/dataset/imagenet/imagenet_64x64/' + org for org in original][:18379]
#infer = os.listdir('Image_Processing/Image_Classifier/zhang_2016/results_total/landscape/results/') # Just to get outputs names
infer = os.listdir('Image_Processing/Image_Classifier/zhang_2016/results_total/imagenet/results/')
models = ['ddcolor', 'larsson_2016', 'siggraph_2016', 'zhang_2016']

In [22]:
total_CPSNR_2_images(original[0], 'Image_Processing/Image_Classifier/zhang_2016/results_total/imagenet/results/'+infer[0])

28.5478725268355

In [34]:
# Testing single image

CPSNR_total = []

org = './Dataset/dataset/landscape_dat/landscape_64x64/00000000_(2).jpg'

print(org)

for model in models:
    inf_complete = f'./Image_Processing/Image_Classifier/{model}/results_total/landscape/results/00000000_(2).jpg'
    try:
        cpsnr = total_CPSNR_2_images(org, inf_complete)
        print(cpsnr)
    except:
        cpsnr = 1000
    CPSNR_total.append(cpsnr)

print(CPSNR_total)

models = ['ddcolor', 'larsson_2016', 'siggraph_2016', 'zhang_2016']

./Dataset/dataset/landscape_dat/landscape_64x64/00000000_(2).jpg
28.070515379780044
27.547093933815713
28.109220359510548
28.0717982604071
[28.070515379780044, 27.547093933815713, 28.109220359510548, 28.0717982604071]


In [44]:
with open('Image_Processing/Image_Classifier/dataset_64x64_cpsnr.csv', 'a', newline='') as csvfile:
    fieldnames = ['datatype', 'image', 'model']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    #writer.writeheader()
    
    for org, inf in zip(original, infer):
        
        CPSNR_total = []
        
        for data in models:
            inf_complete = f'Image_Processing/Image_Classifier/{data}/results_total/imagenet/results/'+ inf
            try:
                cpsnr = total_CPSNR_2_images(org, inf_complete)
            except:
                cpsnr = 1000
                print("here")
            CPSNR_total.append(cpsnr)
        
        print(CPSNR_total)
        x = input("Enter")

        model = models[np.argmin(CPSNR_total)]
        
        print(inf,model)
        #writer.writerow({'datatype':'imagenet', 'image':inf, 'model':model})

[28.621075733938362, 27.579764598050964, 29.00577468250077, 28.5478725268355]
image_00000.jpg larsson_2016
[29.35773799660202, 29.161257538760992, 29.609616909359474, 29.529286151518615]
image_00001.jpg larsson_2016
[28.835828561175568, 28.617761305796144, 29.56377622124853, 29.139085345910694]
image_00002.jpg larsson_2016
[28.495419267573645, 28.541453988975164, 28.35363080157557, 28.470005305080658]
image_00003.jpg siggraph_2016
[28.652079853968658, 27.88049991434216, 28.671082099995683, 28.393220553834126]
image_00004.jpg larsson_2016
[28.686213852227258, 28.627652747884994, 28.881557037606026, 29.019604818509737]
image_00005.jpg larsson_2016
[28.723392305417313, 27.842551630719036, 29.08188972575497, 28.59353772493381]
image_00006.jpg larsson_2016
[28.627704262914225, 28.018519874943944, 29.395613727258848, 29.519329018645976]
image_00007.jpg larsson_2016
[28.737606184471844, 27.830902305387188, 28.09664851310712, 27.92838613280606]
image_00008.jpg larsson_2016
[28.085001157210137,

# CPSNR with 3 models

In [7]:
cd

C:\Users\ivann


c:\Users\ivann\anaconda3\envs\deep_learning\Lib\site-packages\IPython\core\magics\osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
c:\Users\ivann\anaconda3\envs\deep_learning\Lib\site-packages\IPython\core\magics\osm.py:428: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [8]:
cd "Documents\School\8vo. Semestre\Deep Learning"

C:\Users\ivann\Documents\School\8vo. Semestre\Deep Learning


c:\Users\ivann\anaconda3\envs\deep_learning\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [11]:
import cv2
import csv
from skimage.color import rgb2lab
import numpy as np
from tqdm import tqdm
import os
import math

def total_CPSNR_2_images(image1_path, image_2_path):
    # Load images
    image1 = cv2.imread(image1_path)
    image2 = cv2.imread(image_2_path)

    mse = (np.sum(np.square(image1-image2)))/(3*image1.shape[0]*image1.shape[1])
    div = (255**2)/(mse)
    cpsnr = 10*math.log10(div)

    return cpsnr

original = os.listdir('Dataset/dataset/landscape_dat/landscape_64x64/')
#original = os.listdir('Dataset/dataset/imagenet/imagenet_64x64/')
original = ['Dataset/dataset/landscape_dat/landscape_64x64/' + org for org in original]#[:18379]
#original = ['Dataset/dataset/imagenet/imagenet_64x64/' + org for org in original][:18379]
infer = os.listdir('Image_Processing/Image_Classifier/zhang_2016/results_total/landscape/results/') # Just to get outputs names
#infer = os.listdir('Image_Processing/Image_Classifier/zhang_2016/results_total/imagenet/results/')
models = ['ddcolor', 'siggraph_2016', 'zhang_2016']

In [ ]:
total_CPSNR_2_images(original[0], 'Image_Processing/Image_Classifier/zhang_2016/results_total/imagenet/results/'+infer[0])

In [12]:
with open('Image_Processing/Image_Classifier/dataset_64x64_cpsnr_3mod.csv', 'a', newline='') as csvfile:
    fieldnames = ['datatype', 'image', 'model']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    #writer.writeheader()
    
    for org, inf in zip(original, infer):
        
        CPSNR_total = []
        
        for data in models:
            #inf_complete = f'Image_Processing/Image_Classifier/{data}/results_total/imagenet/results/'+ inf
            inf_complete = f'Image_Processing/Image_Classifier/{data}/results_total/landscape/results/'+ inf
            try:
                cpsnr = total_CPSNR_2_images(org, inf_complete)
            except:
                cpsnr = 1000
                print("here")
            CPSNR_total.append(cpsnr)
        
        print(CPSNR_total)
        #x = input("Enter")

        model = models[np.argmin(CPSNR_total)]
        
        print(inf,model)
        writer.writerow({'datatype':'landscape', 'image':inf, 'model':model})

[27.60820581257932, 28.629563175462522, 28.661039449156203]
00000000.jpg ddcolor
[28.070515379780044, 28.109220359510548, 28.0717982604071]
00000000_(2).jpg ddcolor
[28.802610661692924, 27.6238675408243, 27.9126510095799]
00000000_(3).jpg siggraph_2016
[29.78548105694079, 32.05620101688447, 30.078999754027244]
00000000_(4).jpg ddcolor
[27.9397286397743, 27.989500689918554, 27.92152004993881]
00000000_(5).jpg zhang_2016
[30.064053152785473, 32.014367330872915, 30.235792924504374]
00000000_(6).jpg ddcolor
[29.26728734872185, 32.40105010132828, 30.151457055583254]
00000000_(7).jpg ddcolor
[28.12562188666394, 28.27301915463243, 28.24114533551429]
00000001.jpg ddcolor
[28.22646979329673, 28.536150597948556, 28.143932141772364]
00000001_(2).jpg zhang_2016
[28.36899329512707, 30.466525053540522, 29.436671880939237]
00000001_(3).jpg ddcolor
[29.163955774545496, 29.857324413866838, 29.911136340926042]
00000001_(4).jpg ddcolor
[29.34191172564757, 29.521704186050265, 29.653150649956054]
00000001_